In [4]:
import numpy as np
import pandas as pd

# Przygotowanie danych
splits_1 = {'train': 'train.csv', 'validation': 'dev.csv', 'test': 'test.csv'}
df_1 = pd.read_csv("hf://datasets/Adapting/empathetic_dialogues_v2/" + splits_1["train"])

splits_2 = {'train': 'train.jsonl', 'validation': 'validation.jsonl', 'test': 'test.jsonl'}
df_2 = pd.read_json("hf://datasets/SetFit/emotion/" + splits_2["train"], lines=True)

df_1.drop(columns=['id', 'chat_history', 'sys_response', 'question or not', 'behavior'], inplace=True)
df_2.drop(columns=['label'], inplace=True)

df_1.rename(columns={'situation': 'text'}, inplace=True)
df_2.rename(columns={'label_text': 'emotion'}, inplace=True)

combined_df = pd.concat([df_1, df_2])
combined_df

,text,emotion
0,I have always been a big fan of childrens plac...,faithful
1,I am piss-off,angry
2,I am feeling happy I can afford what I need fo...,joyful
3,I saw my friend projectile vomit on the roller...,disgusted
4,I was very happy when I got to see my aunt at ...,grateful
...,...,...
15995,i just had a very brief time in the beanbag an...,sadness
15996,i am now turning and i feel pathetic that i am...,sadness
15997,i feel strong and good overall,joy
15998,i feel like this was such a rude comment and i...,anger


In [5]:
import torch
print("Czy CUDA jest dostępna:", torch.cuda.is_available())
print("Nazwa GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Brak GPU")

torch.set_float32_matmul_precision('medium')

Czy CUDA jest dostępna: True
Nazwa GPU: NVIDIA GeForce RTX 4070 Ti SUPER


In [6]:
from autogluon.multimodal import MultiModalPredictor
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(combined_df, test_size=0.2, random_state=42)

predictor = MultiModalPredictor(label='emotion')
predictor.fit(train_df)

performance = predictor.evaluate(test_df)
print("Wyniki modelu:\n", performance)

performance

No path specified. Models will be saved in: "AutogluonModels\ag-20241112_165247"
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.8.20
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          12
Pytorch Version:    2.4.1
CUDA Version:       12.4
Memory Avail:       17.25 GB / 31.62 GB (54.6%)
Disk Space Avail:   344.71 GB / 797.50 GB (43.2%)
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	First 10 (of 38) unique label values:  ['terrified', 'sentimental', 'trusting', 'disgusted', 'annoyed', 'sadness', 'anger', 'love', 'content', 'joy']
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression', 'quantile'])

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 0, global step 166: 'val_accuracy' reached 0.33680 (best 0.33680), saving model to 'E:\\Projekty\\DataSpellProjects\\SUML_Emotions\\AutogluonModels\\ag-20241112_165247\\epoch=0-step=166.ckpt' as top 3


...

Predicting: |          | 0/? [00:00<?, ?it/s]

D:\Programy\Anaconda3\envs\autogluon\lib\site-packages\autogluon\multimodal\utils\checkpoint.py:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(per

Predicting: |          | 0/? [00:00<?, ?it/s]

Wyniki modelu:
 {'accuracy': 0.9076362343319406}


{'accuracy': 0.9076362343319406}